In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('stackoverflow-reduced.csv')


In [3]:
data.head()

,post,tags
0,php domnode nodevalue &gt; &lt issue i am try...,php
1,problems running a php script from command lin...,php
2,how to know or ensure that a php page is only ...,php
3,posting child objects to a database with php ...,php
4,how to fix this 2 issue by php (edit user from...,php


In [4]:
print("Input data has {} rows and {} columns".format(len(data), len(data.columns)))

Input data has 6433 rows and 2 columns


In [5]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
from bs4 import BeautifulSoup

stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

In [6]:
def clean_text(text):
    text = BeautifulSoup(text, "lxml").text
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) 
    tokens = re.split('\W+', text)
    text = [wn.lemmatize(word) for word in tokens if word not in stopwords]
    return text

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['post'], data['tags'], test_size=0.2)

In [8]:
tfidf_vect = TfidfVectorizer(analyzer=clean_text, min_df=2, encoding='latin-1', ngram_range=(1, 2))
tfidf_vect_fit = tfidf_vect.fit(X_train)

tfidf_train = tfidf_vect_fit.transform(X_train)
tfidf_test = tfidf_vect_fit.transform(X_test)

X_train_vect = pd.DataFrame(tfidf_train.toarray())
X_test_vect = pd.DataFrame(tfidf_test.toarray())
X_train_vect.head()

,0,1,2,3,4,5,6,7,8,9,...,10633,10634,10635,10636,10637,10638,10639,10640,10641,10642
0,0.036166,0.093250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.034864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.058742,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.072132,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [10]:
import xgboost as xgb


ModuleNotFoundError: No module named 'xgboost'

In [31]:
xgb = xgb.XGBClassifier(objective="multi:softprob", random_state=42, n_estimators=150, max_depth=11)

xgb_model = xgb.fit(X_train_vect, y_train)
y_pred = xgb_model.predict(X_test_vect)

precision, recall, fscore, train_support = score(y_test, y_pred)
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
accuracy = ((y_pred==y_test).sum()/len(y_pred))
print('Accuracy: {}'.format(accuracy))
    

C:\Users\Swetha\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Precision: [0.95067265 0.97326203 0.98421053]
Recall: [0.96803653 0.95789474 0.97905759]
Accuracy: 0.9683333333333334
